# SciPy

SciPy is a collection of numerical algorithms with python interfaces.  In many cases, these interfaces are wrappers around standard numerical libraries that have been developed in the community and are used with other languages.  Usually detailed references are available to explain the implementation.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

<div class="alert alert-block alert-warning">

There are many subpackages.  Generally, you load the subpackages separately, e.g.

```
    
 from scipy import linalg, optimize
    
    
```
then you have access to the methods in those namespaces
</div>

## Numerical Methods 

One thing to keep in mind -- all numerical methods have strengths and weaknesses, and make assumptions.  You should always do some research into the method to understand what it is doing.

It is also always a good idea to run a new method on some test where you know the answer, to make sure it is behaving as expected.

## Integration

we'll do some integrals of the form

$$I = \int_a^b f(x) dx$$

We can imagine two situations:
* our function $f(x)$ is given by an analytic expression.  This gives us the freedom to pick our integration points, and in general can allow us to optimize our result and get high accuracy
* our function $f(x)$ is defined on at a set of (possibly regular spaced) points.  

In numerical analysis, the term _quadrature_ is used to describe any integration method that represents the integral as the weighted sum of a discrete number of points.

In [ ]:
from scipy import integrate

quad is the basic integrator for a general (not sampled) function.  It uses a general-interface from the Fortran package QUADPACK (QAGS or QAGI).  It will return the integral in an interval and an estimate of the error in the approximation

In [ ]:
def f(x):
    return np.sin(x)**2

In [ ]:
I, err = integrate.quad(f, 0.0, 2.0*np.pi)
print(I)
print(err)

We can ask for a more accurate integral by setting the absolute and relative tolerances, `epsabs` and `epsrel`

In [ ]:
I, err = integrate.quad(f, 0.0, 2.0*np.pi, epsabs=1.e-14, epsrel=1.e-14)
print(I)
print(err)

In [ ]:
help(integrate.quad)

sometimes our integrand function takes optional arguments

In [ ]:
def g(x, A, sigma):
    return A*np.exp(-x**2/sigma**2)

In [ ]:
I, err = integrate.quad(g, -1.0, 1.0, args=(1.0, 2.0))
print(I, err)

numpy defines the inf quantity which can be used in the integration limits.  We can integrate a Gaussian (we know the answer is $\sqrt{\pi}$).

Note: behind the scenes, what the integration function does is do a variable transform like: $t = 1/x$.  This works when one limit is $\infty$, giving

$$\int_a^b f(x) dx = \int_{1/b}^{1/a} \frac{1}{t^2} f\left (\frac{1}{t}\right) dt$$

In [ ]:
I, err = integrate.quad(g, -np.inf, np.inf, args=(1.0, 1.0))
print(I, err)

### integration of a sampled function

Here we integrate a function that is defined only at a sequece of points.  Let's compute

$$I = \int_0^{2\pi} f(x_i) dx$$

with $x_i = 0, \ldots, 2\pi$ defined at $N$ points

In [ ]:
N = 17
x = np.linspace(0.0, 2.0*np.pi, N, endpoint=True)
y = np.sin(x)**2

I = integrate.simps(y, x)
print(I)

<div class="alert alert-block alert-info"><h3><span class="fa fa-flash"></span> Quick Exercise:</h3>

We know the answer for this intergral, so we can look at how the error changes with the number of samples.  Vary $N$ and plot the absolute value of the error in the solution vs. N (the error for this integral is: $\epsilon = |I - \pi|$).  How does it behave?

## Root Finding

Often we need to find a value of a variable that zeros a function -- this is _root finding_.  Sometimes, this is a multidimensional problem.

The `brentq()` routine offers a very robust method for find roots from a scalar function.  You do need to provide an interval that bounds the root.

$f(x) = \frac{x e^x}{e^x - 1} - 5$

In [ ]:
import scipy.optimize as optimize

def f(x):
    # this is the non-linear equation that comes up in deriving Wien's law for radiation
    return (x*np.exp(x)/(np.exp(x) - 1.0) - 5.0)

root, r = optimize.brentq(f, 0.1, 10.0, full_output=True)

print(root)
print(r.converged)

In [ ]:
x = np.linspace(0.1, 10.0, 1000)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, f(x))
ax.plot(np.array([root]), np.array([f(root)]), 'ro')

## ODEs

Many methods exist for integrating ordinary differential equations.  Most will want you to write your ODEs as a system of first order equations.

This system of ODEs is the Lorenz system:

$$\frac{dx}{dt} = \sigma (y - x)$$
$$\frac{dy}{dt} = rx - y - xz$$
$$\frac{dz}{dt} = xy - bz$$

In [ ]:
# system parameters
sigma = 10.0
b = 8./3.
r = 28.0

def rhs(t, X):
    x, y, z = X
    xdot = sigma*(y - x)
    ydot = r*x - y - x*z
    zdot = x*y - b*z

    return np.array([xdot, ydot, zdot])

We'll use the `solve_ivp` function to solve this.  The function below is a wrapper to it our problem, taking the initial conditions, time interval at which we want the solution stored (`dt`) and maximum time to integrate to (`tmax`)

In [ ]:
def ode_integrate(X0, dt, tmax):
    """ integrate using the VODE method, storing the solution each dt """

    r = integrate.solve_ivp(rhs, (0.0, tmax), X0,
                            method="RK45", dense_output=True)

    # get the solution at intermediate times
    ts = np.arange(0.0, tmax, dt)
    
    Xs = r.sol(ts)
    return ts, Xs

Now let's integrate the system and plot the solution

In [ ]:
t, X = ode_integrate([1.0, 1.0, 20.0], 0.02, 30)

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(X[0,:], X[1,:], X[2,:])
fig.set_size_inches(8.0,6.0)

<div class="alert alert-block alert-info"><h3><span class="fa fa-flash"></span> Quick Exercise:</h3>

This problem was what lead to the discovery of chaos in non-linear systems --- widely different solutions arising from small differences in the initial conditions.
    
To see this, start with two points that are very close together (e.g., add 1.e-8 to the initial x position) and integrate both sets of initial conditions and make plots showing $x$ vs. $t$, $y$ vs. t, and $z$ vs. t.
</div>

### Multi-variate root find

The steady states of this system correspond to:

$${\bf f}({\bf x}) = 
\left ( \begin{array}{c}
\sigma (y -x) \\ 
rx - y -xz \\
xy - bz \\
\end{array}\right )
= 0$$


We want to find the value ${\bf x}$ that zeros this vector function.  These methods usually require a Jacobian matrix, so we'll construct that

In [ ]:
def jac(t, x):

    return np.array(
        [ [-sigma, sigma, 0.0], 
          [r - x[2], -1.0, -x[0]],
          [x[1], x[0], -b] ])

The optimize methods also will want a single function that returns the function and its Jacobian:

In [ ]:
def f(x):
    return rhs(0.,x), jac(0.,x)

we can find the steady points in this system by doing a multi-variate root find on the RHS vector

In [ ]:
sol1 = optimize.root(f, [1., 1., 1.], jac=True)
print(sol1.x)

sol2 = optimize.root(f, [10., 10., 10.], jac=True)
print(sol2.x)

sol3 = optimize.root(f, [-10., -10., -10.], jac=True)
print(sol3.x)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(X[0,:], X[1,:], X[2,:])

ax.scatter(sol1.x[0], sol1.x[1], sol1.x[2], marker="x", color="r")
ax.scatter(sol2.x[0], sol2.x[1], sol2.x[2], marker="x", color="r")
ax.scatter(sol3.x[0], sol3.x[1], sol3.x[2], marker="x", color="r")

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")


## Fitting

Fitting is used to match a model to experimental data.  E.g. we have N points of $(x_i, y_i)$ with associated errors, $\sigma_i$, and we want to find a simple function that best represents the data.

Usually this means that we will need to define a metric, often called the *residual*, for how well our function matches the data, and then minimize this residual.  Least-squares fitting is a popular formulation.

We want to fit our data to a function $Y(x, \{a_j\})$, where $a_j$ are model parameters we can adjust.  We want to find the optimal $a_j$ to minimize the distance of $Y$ from our data:
$$\Delta_i = Y(x_i, \{a_j\}) - y_i$$

Least-squares minimizes $\chi^2$:
$$\chi^2(\{a_j\}) = \sum_{i=1}^N \left ( \frac{\Delta_i}{\sigma_i} \right )^2$$

### general linear least squares

First we'll make some experimental data (a quadratic with random fashion).  We use the `randn()` function to provide Gaussian normalized errors.

In [ ]:
def y_experiment2(a1, a2, a3, sigma, x):
    """ return the experimental data in a quadratic + random fashion,                              
        with a1, a2, a3 the coefficients of the quadratic and sigma is                             
        the error.  This will be poorly matched to a linear fit for                                
        a3 != 0 """

    N = len(x)

    # randn gives samples from the "standard normal" distribution                                  
    r = np.random.randn(N)

    y = a1 + a2*x + a3*x*x + sigma*r

    return y

N = 40
sigma = 5.0*np.ones(N)

x = np.linspace(0, 100.0, N)
y = y_experiment2(2.0, 1.50, -0.02, sigma, x)

plt.scatter(x,y)
plt.errorbar(x, y, yerr=sigma, fmt="o")

In [ ]:
def resid(avec, x, y, sigma):
    """ the residual function -- this is what will be minimized by the
        scipy.optimize.leastsq() routine.  avec is the parameters we
        are optimizing -- they are packed in here, so we unpack to
        begin.  (x, y) are the data points 

        scipy.optimize.leastsq() minimizes:

           x = arg min(sum(func(y)**2,axis=0))
                    y

        so this should just be the distance from a point to the curve,
        and it will square it and sum over the points
        """

    a0, a1, a2 = avec
    
    return (y - (a0 + a1*x + a2*x**2))/sigma


# initial guesses
a0, a1, a2 = 1, 1, 1

afit, flag = optimize.leastsq(resid, [a0, a1, a2], args=(x, y, sigma))

print(afit)

plt.plot(x, afit[0] + afit[1]*x + afit[2]*x*x )
plt.scatter(x,y)
plt.errorbar(x, y, yerr=sigma, fmt="o")

$\chi^2$

In [ ]:
chisq = sum(np.power(resid(afit, x, y, sigma),2))
normalization = len(x)-len(afit)
print(chisq/normalization)

### a nonlinear example

our experiemental data -- an exponential

In [ ]:
a0 = 2.5
a1 = 2./3.
sigma = 5.0

a0_orig, a1_orig = a0, a1

x = np.linspace(0.0, 4.0, 25)
y = a0*np.exp(a1*x) + sigma*np.random.randn(len(x))

plt.scatter(x,y)
plt.errorbar(x, y, yerr=sigma, fmt="o", label="_nolegend_")

our function to minimize

In [ ]:
def resid(avec, x, y):
    """ the residual function -- this is what will be minimized by the                             
        scipy.optimize.leastsq() routine.  avec is the parameters we                               
        are optimizing -- they are packed in here, so we unpack to                                 
        begin.  (x, y) are the data points                                                         
                                                                                                   
        scipy.optimize.leastsq() minimizes:                                                        
                                                                                                   
           x = arg min(sum(func(y)**2,axis=0))                                                     
                    y                                                                              
                                                                                                   
        so this should just be the distance from a point to the curve,                             
        and it will square it and sum over the points                                              
        """

    a0, a1 = avec

    # note: if we wanted to deal with error bars, we would weight each                             
    # residual accordingly                                                                         
    return y - a0*np.exp(a1*x)

In [ ]:
a0, a1 = 1, 1
afit, flag = optimize.leastsq(resid, [a0, a1], args=(x, y))

print(flag)
print(afit)

In [ ]:
plt.plot(x, afit[0]*np.exp(afit[1]*x),
           label=r"$a_0 = $ %f; $a_1 = $ %f" % (afit[0], afit[1]))
plt.plot(x, a0_orig*np.exp(a1_orig*x), ":", label="original function")
plt.legend(numpoints=1, frameon=False)
plt.scatter(x,y, c="k")
plt.errorbar(x, y, yerr=sigma, fmt="o", label="_nolegend_")

## FFTs

Fourier transforms convert a physical-space (or time series) representation of a function into frequency space.  This provides an equivalent representation of the data with a new view.

The FFT and its inverse in NumPy use:
$$F_k = \sum_{n=0}^{N-1} f_n e^{-2\pi i nk/N}$$

$$f_n = \frac{1}{N} \sum_{k=0}^{N-1} F_k 
   e^{2\pi i n k/N}$$
   

Both NumPy and SciPy have FFT routines that are similar.  However, the NumPy version returns the data in a more convenient form.

It's always best to start with something you understand -- let's do a simple sine wave.  Since our function is real, we can use the rfft routines in NumPy -- the understand that we are working with real data and they don't return the negative frequency components.

One important caveat -- FFTs assume that you are periodic.  If you include both endpoints of the domain in the points that comprise your sample then you will not match this assumption.  Here we use endpoint=False with linspace()

We'll start by considering 

$$ f(x) = \sin(2\pi f_0 x)$$

Here f_0 is the *wavenumber* with units of 1 / length (or 1 / time if we were instead considering $f(t)$)

The wavelength is the $1/f_0$.

In [ ]:
def single_freq_sine(npts):

    # a pure sine with no phase shift will result in pure imaginary                                         
    # signal                                                                                                
    f_0 = 0.2

    # we'll make our domain contain 10 wavelengths
    xmax = 10.0/f_0
    
    xx = np.linspace(0.0, xmax, npts, endpoint=False)

    f = np.sin(2.0*np.pi*f_0*xx)

    return xx, f

To make our life easier, we'll define a function that plots all the stages of the FFT process

In [ ]:
def plot_FFT(xx, f):

    npts = len(xx)

    # Forward transform: f(x) -> F(k)                                                                       
    fk = np.fft.rfft(f)

    # Normalization -- the '2' here comes from the fact that we are                                         
    # neglecting the negative portion of the frequency space, since                                         
    # the FFT of a real function contains redundant information, so                                         
    # we are only dealing with 1/2 of the frequency space.                                                  
    #                                                                                                       
    # technically, we should only scale the 0 bin by N, since k=0 is                                        
    # not duplicated -- we won't worry about that for these plots                                           
    norm = 2.0/npts

    fk = fk*norm

    fk_r = fk.real
    fk_i = fk.imag

    # the fftfreq returns the postive and negative (and 0) frequencies                                      
    # the newer versions of numpy (>=1.8) have an rfftfreq() function                                       
    # that really does what we want -- we'll use that here.                                                 
    k = np.fft.rfftfreq(npts)

    # to make these dimensional, we need to divide by dx.  
    dx = xx[1] - xx[0]
    kfreq = k / dx

    # Inverse transform: F(k) -> f(x) -- without the normalization                                          
    fkinv = np.fft.irfft(fk/norm)

    # plots
    fig = plt.figure()
    ax1 = fig.add_subplot(411)
    
    ax1.plot(xx, f)
    ax1.set_xlabel("x")
    ax1.set_ylabel("f(x)")

    ax2 = fig.add_subplot(412)

    ax2.plot(kfreq, fk_r, label=r"Re($\mathcal{F}$)")
    ax2.plot(kfreq, fk_i, ls=":", label=r"Im($\mathcal{F}$)")
    ax2.set_xlabel(r"$\nu_k$")
    ax2.set_ylabel("F(k)")

    ax2.legend(fontsize="small", frameon=False)

    ax3 = fig.add_subplot(413)

    ax3.plot(kfreq, np.abs(fk))
    ax3.set_xlabel(r"$\nu_k$")
    ax3.set_ylabel(r"|F(k)|")

    ax4 = fig.add_subplot(414)

    ax4.plot(xx, fkinv.real)
    ax4.set_xlabel(r"$\nu_k$")
    ax4.set_ylabel(r"inverse F(k)")

    fig.set_size_inches(10,8)
    plt.tight_layout()


In [ ]:
npts = 128
xx, f = single_freq_sine(npts)
plot_FFT(xx, f)

A cosine is shifted in phase by pi/2

In [ ]:
def single_freq_cosine(npts):

    # a pure cosine with no phase shift will result in pure real                                            
    # signal                                                                                                
    f_0 = 0.2

    xmax = 10.0/f_0

    xx = np.linspace(0.0, xmax, npts, endpoint=False)

    f = np.cos(2.0*np.pi*f_0*xx)

    return xx, f

In [ ]:
xx, f = single_freq_cosine(npts)
plot_FFT(xx, f)

Now let's look at a sine with a pi/4 phase shift

In [ ]:
def single_freq_sine_plus_shift(npts):

    # a pure sine with no phase shift will result in pure imaginary                                         
    # signal                                                                                                
    f_0 = 0.2

    xmax = 10.0/f_0

    xx = np.linspace(0.0, xmax, npts, endpoint=False)

    f = np.sin(2.0*np.pi*f_0*xx + np.pi/4)

    return xx, f

In [ ]:
xx, f = single_freq_sine_plus_shift(npts)
plot_FFT(xx, f)

### A frequency filter

we'll setup a simple two-frequency sine wave and filter a component

In [ ]:
def two_freq_sine(npts):

    # a pure sine with no phase shift will result in pure imaginary             
    # signal                                                                    
    f_0 = 0.2
    f_1 = 0.5

    xmax = 10.0/f_0

    # we call with endpoint=False -- if we include the endpoint, then for       
    # a periodic function, the first and last point are identical -- this       
    # shows up as a signal in the FFT.                                          
    xx = np.linspace(0.0, xmax, npts, endpoint=False)

    f = 0.5*(np.sin(2.0*np.pi*f_0*xx) + np.sin(2.0*np.pi*f_1*xx))

    return xx, f

In [ ]:
npts = 256

xx, f = two_freq_sine(npts)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(xx, f)

we'll take the transform: f(x) -> F(k)

In [ ]:
# normalization factor: the 2 here comes from the fact that we neglect          
# the negative portion of frequency space because our input function            
# is real                                                                       
norm = 2.0/npts
fk = norm*np.fft.rfft(f)

ofk_r = fk.real.copy()
ofk_i = fk.imag.copy()

# get the frequencies
k = np.fft.rfftfreq(len(xx))

# normalize using dx
# This makes the frequencies essentially multiples of 1/dx
kfreq = k / (x[1] - x[0])

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(kfreq, fk.real, label="real")
ax.plot(kfreq, fk.imag, ":", label="imaginary")
ax.legend(frameon=False)

Filter out the higher frequencies

In [ ]:
fk[kfreq > 0.4] = 0.0

# element 0 of fk is the DC component                                           
fk_r = fk.real
fk_i = fk.imag


# Inverse transform: F(k) -> f(x)                                               
fkinv = np.fft.irfft(fk/norm)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(xx, fkinv.real)

## Linear Algebra

### general manipulations of matrices

you can use regular NumPy arrays and the `@` matrix multiplication operator

In [ ]:
a = np.array([[1.0, 2.0], [3.0, 4.0]])

In [ ]:
print(a)
print(a.transpose())
print(a.T)

In [ ]:
ainv = np.linalg.inv(a)
print(ainv)

In [ ]:
a @ ainv

the eye() function will generate an identity matrix (as will the identity())

In [ ]:
print(np.eye(2))
print(np.identity(2))

we can solve Ax = b

In [ ]:
b = np.array([5, 7])
x = np.linalg.solve(a, b)
print(x)